In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
import requests

# Define the API URL
api_url = 'https://newsapi.org/v2/everything'
api_key = '' #insert API key here

# Define the query parameters
params = {
    'q': 'climate change',
    'apiKey': api_key
}

# Send a GET request to the API
response = requests.get(api_url, params=params)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Access the response content (assuming it's in JSON format)
    data = response.json()

    # Extract the articles from the response
    articles = data['articles']

    # Format the article data
    formatted_articles = []
    for article in articles:
        author = article['author']
        title = article['title']
        description = article['description']
        url = article['url']

        # Format the article details
        formatted_article = f"Author: {author}\nTitle: {title}\nDescription: {description}\nURL: {url}\n"
        formatted_articles.append(formatted_article)

    print(formatted_articles)
else:
    # Request was not successful
    print(f"Error: {response.status_code}")


['Author: Matt Simon\nTitle: Enjoy Your Favorite Wine Before Climate Change Destroys It\nDescription: Extreme heat and droughts are making it harder to grow grapes in many traditional regions. Here’s how scientists are helping the industry adapt.\nURL: https://www.wired.com/story/enjoy-your-favorite-wine-before-climate-change-destroys-it/\n', 'Author: Justine Calma\nTitle: Can brightening clouds buy us time to fight climate change?\nDescription: Scientists proposed a road map for studying solar geoengineering as a potential solution to climate change. Manipulating clouds might be able to cool down the planet, but there could be unintended consequences.\nURL: https://www.theverge.com/2024/3/21/24107762/marine-cloud-brightening-climate-change-science-advances\n', 'Author: Justine Calma\nTitle: Babies born this year face a $500,000 climate bill\nDescription: A new study commissioned by Consumer Reports estimates rising cost of living expenses with climate change for a baby born in 2024.\n

This uses a **sentence-transformers model**: It maps sentences & paragraphs to a 768 dimensional dense vector space and can be used for tasks like clustering or semantic search.

In [ ]:
#installing and importing libraries

!pip install sentence_transformers

from sentence_transformers import SentenceTransformer
import numpy as np
import csv
import pandas as pd
import scipy


# choose LLM model to activate with SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

# Define your corpus

corpus = formatted_articles

# Convert your Corpus into vectors

corpus_embeddings = model.encode(corpus)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#index the corpus with cosine similarity

queries = ["fires and water issues"] # The query for semantic search
query_embeddings = model.encode(queries)

# creating search interface with cosine distance
closest_n = 10
articles=[]
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 10 most relevant articles for your query:")
    for idx, distance in results[0:closest_n]:
        print("(Score: %.4f)" % (1-distance), corpus[idx].strip(), "|| article index:", idx )
        articles.append(corpus[idx])





Query: fires and water issues

Top 10 most relevant articles for your query:
(Score: 0.4203) Author: Angela Haupt
Title: Why So Many Dogs Have Allergies Now
Description: More dogs are suffering from atopic dermatitis, otherwise known as environmental allergies. Climate change plays a big role.
URL: https://time.com/6961113/dog-allergies-atopic-dematitis/ || article index: 17
(Score: 0.4108) Author: Emmanuel Akinwotu
Title: Lethal heat in West Africa is driven by human-caused climate change
Description: The recent deadly heat in West Africa is driven by human activities, including the burning of fossil fuels, particularly in the wealthy Northern Hemisphere, according to an international report.
URL: https://www.npr.org/2024/04/18/1245524216/west-africa-heatwave-climate-change || article index: 97
(Score: 0.4104) Author: BeauHD
Title: Methane From Landfills Is a Big Driver of Climate Change, Study Says
Description: An anonymous reader quotes a report from the New York Times: They're 

In [ ]:
import json

print (results)

# Convert the results dictionary to JSON format
json_results = json.dumps(results)

# Convert the JSON string to a Python list
result_list = json.loads(json_results)

# print (result_list)
# for i in range(len(result_list)):
#   print (corpus[result_list[i][0]])
# # Extract the relevant values from the result_list
# articles = [corpus[idx] for idx in result_list[0]]

# Create the userInput variable to pass to the GPT api

query = "<where are the fires and water issues?>"
userInput = query + ' '.join(articles)

[(17, 0.5797169114344836), (97, 0.5892268744821773), (13, 0.5896158299629595), (70, 0.6163475724316809), (72, 0.620452456566748), (38, 0.6250960666866558), (35, 0.6308433291904649), (93, 0.6366972388623158), (63, 0.6373983729662047), (76, 0.6443392867322899), (28, 0.6445812228501715), (22, 0.6476893769374219), (89, 0.6505584536630142), (24, 0.650926302446525), (31, 0.6515558814703273), (71, 0.6573866014949246), (94, 0.6581632229935047), (61, 0.6638463500595093), (37, 0.6657690510577177), (91, 0.6659739565417788), (11, 0.6677209040540126), (19, 0.6694251856483389), (7, 0.6727538255411357), (56, 0.6727538255411357), (82, 0.6779617005396474), (64, 0.6845661857865364), (65, 0.6881671730555412), (77, 0.689635360189318), (80, 0.691330840386487), (20, 0.692596771079795), (15, 0.6944236909744566), (9, 0.698097920340797), (75, 0.7060757355438275), (1, 0.7102560898403769), (30, 0.7114234197865392), (14, 0.7132368150171773), (21, 0.7170505650665135), (3, 0.719699646670274), (8, 0.719830374557876)

In [ ]:
from IPython.core.display import display, HTML

prompt = """""

You are a reading assistant that writes a news brief for user <query>, based on
a list of  ^Titles^. For every sentence you write, cite the  ^Titles^ name.

At the end of your answer:
1. Add keywords from the  ^Titles^.
2. Create a sources list of each result you cited, with the  ^Titles^ name,
author, and a link in HTML tag format with _blank.

If the Titles are not related to the query, tell that to the reader.

"""

def fetch_chat_completion(userInput):
    api_url = 'https://api.openai.com/v1/chat/completions'
    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer ', #insert API key here
    }

    body = {
        "model": "gpt-4",
        "messages": [
            {
                "role": "system",
                "content": prompt
            },
            {
                "role": "user",
                "content": userInput
            }
        ]
    }


    try:
      response = requests.post(api_url, headers=headers, data=json.dumps(body))
      response.raise_for_status()
      chat_completion_answer = response.json()['choices'][0]['message']['content'].replace('\\n', '<br>')
      print (chat_completion_answer)
    except requests.exceptions.HTTPError as err:
      print(f"Error calling the API: {err}")


display(HTML(fetch_chat_completion(userInput)))

Climate change is having a significant impact globally, with noticeable effects in a variety of areas. In regions like West Africa, lethal heat is being driven by human activities such as the burning of fossil fuels, which emphasizes the link between climate change and fire related issues ("Lethal heat in West Africa is driven by human-caused climate change", Emmanuel Akinwotu). Simultaneously, climate change is influencing other natural elements. For example, the frequency of extreme cold upwellings, which can be catastrophic to species like bull sharks, is increasing due to changes in climate ("Deadly upwellings of cold water pose threat to migratory sharks", Madeleine Cuff). 

Human activities are not only direct causes but also indirect contributors to climate change by producing environmental hazards like excessive waste ("Methane From Landfills Is a Big Driver of Climate Change, Study Says", BeauHD) and releasing 'forever chemicals' into drinking water resources ("The US finalize

<IPython.core.display.HTML object>